# The ensemble approach is to take the data, and train each of the N classifiers on a different random arrangement of 6 landmarks. 

In [2]:
# first get all X and y data from the all_points_folds
import numpy as np 
import pickle, random 
import cv2

X, y = [], []

for file in os.listdir("all_points_folds"): 
    with open(f"all_points_folds/{file}", 'rb') as f: 
        X_y = pickle.load(f)
        X.append(X_y[0])
        y.append(X_y[1]) 

In [3]:
import random 
def shuffle(X, y, seed = None):
    if seed == None:  
        seed = random.randrange(0, 100)
        print(f"using seed {seed}")
    np.random.seed(seed) 
    new_X = np.concatenate([X_i for X_i in X])
    new_y = np.concatenate([y_i for y_i in y])
    N = np.random.permutation(new_X.shape[0])
    new_X = new_X[N]
    new_y = new_y[N]
    new_X = new_X.reshape(5, 20, 90, 126)
    new_y = new_y.reshape(5, 20)
    return new_X, new_y
X, y = shuffle(X, y, seed = 68)

# We will now get the landmarks for each of the NUM_MODELS models. 

In [4]:
import random 
NUM_LANDMARKS = 3 
NUM_MODELS = 50 

model_to_arrangements = {} 
for i in range(NUM_MODELS): 
    model_to_arrangements[i] = random.sample(list(range(21)), NUM_LANDMARKS) 

# After getting all configuration, create a function to take in a generic dataset and get the customized one for a model. 

In [5]:
def specify_data(dataset, landmarks): 
    """
    dataset should contain a 3D matrix of (N, 90, 126). 
    """ 
    cols = []
    for landmark in landmarks:
        cols += (np.array([0, 21, 42, 63, 84, 105]) + landmark).tolist()
    
    return dataset[:, :, tuple(cols)]  

# next we'll want to get X_train, X_test, y_train, y_test all ready for creating the models 

In [6]:
VALIDATION_FOLD_INDEX = 0 # index of the fold to be used for validation
X_test, y_test = X[VALIDATION_FOLD_INDEX], y[VALIDATION_FOLD_INDEX]
X_train = np.delete(X, VALIDATION_FOLD_INDEX, axis=0).reshape(80, 90, 126) 
y_train = np.delete(y, VALIDATION_FOLD_INDEX, axis=0).reshape(80,)

In [7]:
import tensorflow as tf 
def make_model(): 
    # use this function to create all of the models 
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=False), 
        tf.keras.layers.Dropout(0.3), 
        tf.keras.layers.Dense(1, activation = "sigmoid")
    ])

    model.compile(loss = "binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    
    return model 

In [8]:
import os

checkpoint = tf.keras.callbacks.ModelCheckpoint("best_ensemble.h5", save_best_only=True, monitor = "val_accuracy")
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", patience=10)
def train_models(): 
    # train all of the models and get them back 
    models, val_accuracies = [], []
    for _, model_configuration in model_to_arrangements.items(): 
        train_data, test_data = specify_data(X_train, model_configuration), specify_data(X_test, model_configuration)
        model = make_model() 

        try: 
            os.remove("best_ensemble.h5") 
        except Exception as e: 
            pass 

        # train this model 
        model.fit(train_data, y_train, validation_data = (test_data, y_test), epochs = 75, callbacks = [checkpoint, early_stopping]) 

        # revert to best weights, if created 
        try: 
            model.load_weights("best_ensemble.h5")
        except Exception as e:
            pass 
        
        loss, accuracy, precision, recall = model.evaluate(test_data, y_test)
        
        val_accuracies.append(accuracy) # add the accuracy 

        models.append(model) # add the model

    return models, np.array(val_accuracies)

In [9]:
models, val_accuracies = train_models() 

Epoch 1/75
3/3 [==============================] - 2s 228ms/step - loss: 0.6858 - accuracy: 0.6125 - precision: 0.5814 - recall: 0.6579 - val_loss: 0.6819 - val_accuracy: 0.8000 - val_precision: 1.0000 - val_recall: 0.6667
Epoch 2/75
3/3 [==============================] - 0s 26ms/step - loss: 0.6811 - accuracy: 0.6875 - precision: 0.6667 - recall: 0.6842 - val_loss: 0.6790 - val_accuracy: 0.7500 - val_precision: 1.0000 - val_recall: 0.5833
Epoch 3/75
3/3 [==============================] - 0s 28ms/step - loss: 0.6837 - accuracy: 0.6125 - precision: 0.5946 - recall: 0.5789 - val_loss: 0.6756 - val_accuracy: 0.7500 - val_precision: 1.0000 - val_recall: 0.5833
Epoch 4/75
3/3 [==============================] - 0s 28ms/step - loss: 0.6772 - accuracy: 0.6875 - precision: 0.6857 - recall: 0.6316 - val_loss: 0.6734 - val_accuracy: 0.7500 - val_precision: 1.0000 - val_recall: 0.5833
Epoch 5/75
3/3 [==============================] - 0s 29ms/step - loss: 0.6731 - accuracy: 0.7500 - precision: 0.800

# We will then need a function to test the model's accuracy on the test set. 
# We will also want to see whether this approach does well on our six videos. 

In [10]:
def aggregate_validation_accuracy(models, X_test, y_test):
    preds = np.zeros((X_test.shape[0], 1))
    for model, landmarks in zip(models, model_to_arrangements.values()):
        test_data = specify_data(X_test, landmarks)
        preds += model.predict(test_data)
    preds = preds / len(models)
    preds = np.round_(preds).flatten()
    return (preds == y_test).mean() 

In [11]:
aggregate_validation_accuracy(models, X_test, y_test)

0.75

In [12]:
import mediapipe as mp 
from tqdm import tqdm 
def hand_locations_general(frame, min_detection_confidence = 0.5, min_tracking_confidence = 0.5): 
    """give all landmarks"""

    hands = mp.solutions.hands.Hands(min_detection_confidence=min_detection_confidence, min_tracking_confidence=min_tracking_confidence) # MAKE SURE THIS IS ALL GOOD 
    results = hands.process(frame.astype('uint8'))
    X_locations = [0] * 42
    Y_locations = [0] * 42
    Z_locations = [0] * 42

    if results.multi_hand_landmarks:
        x = y = z = 0 
        for hand, hand_landmark in enumerate(results.multi_hand_landmarks):
            for i in range(0, 21):
                landmark = hand_landmark.landmark[i]
                X_locations[x] = landmark.x
                Y_locations[y] = landmark.y 
                Z_locations[z] = landmark.z
                x += 1; y += 1; z +=1; 
            
    hands.close()
    return np.concatenate([X_locations, Y_locations, Z_locations]) 


# create a function to pad your videos 
def pad(locations, maxlen = 90, padding = "post", truncating = "post"): 
    new_locations = locations.tolist() 
    empty_row = np.zeros((1, 126))
    for i, video in tqdm(enumerate(new_locations)): 
        if len(video) < maxlen:  
            for new_row in range(maxlen - len(video)): 
                if padding == "post": 
                    new_locations[i] = np.array(new_locations[i])
                    new_locations[i] = np.concatenate([new_locations[i], empty_row])
                if padding == "pre": 
                    new_locations[i] = np.array(new_locations[i])
                    new_locations[i] = np.concatenate([empty_row, new_locations[i]])

        if len(video) > maxlen: 
            if truncating == "post": 
                new_locations[i] = new_locations[i][:maxlen]
            elif truncating == "pre": 
                new_locations[i] = new_locations[i][len(video) - maxlen : ]
    return np.array(new_locations)

In [13]:
# function to ensemble predict on frames. 
def ensemble_predict(models, X_test):
    preds = np.zeros((X_test.shape[0], 1))
    for model, landmarks in tqdm(zip(models, model_to_arrangements.values())):
        test_data = specify_data(X_test, landmarks)
        preds += model.predict(test_data)
    preds = preds / len(models)
    return preds.flatten()

In [13]:
# create a function to then predict on videos 
def predict_on_video(models, path): 
    LOCATIONS = []
    cap = cv2.VideoCapture(path)
    while cap.isOpened():
        _, frame = cap.read()
        if not _: break 

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        LOCATIONS.append(hand_locations_general(frame))
        print("read frame") 
        
    print("read all frames")
    LOCATIONS = np.array([LOCATIONS])
    LOCATIONS = pad(LOCATIONS)
    return ensemble_predict(models, LOCATIONS)

In [13]:
LOCATIONS = []
cap = cv2.VideoCapture("control_1.mov")
while cap.isOpened():
    _, frame = cap.read()
    if not _: break 

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    LOCATIONS.append(hand_locations_general(frame))
    print("read frame") 